In [84]:
import os, sys
import json
from pathlib import Path
import random
from typing import Any, Dict, List
import glob
from google.colab import drive
%cd /content
content = Path('/content')
skol = content / 'drive/My Drive/SKOL'
piggyatbaqaqi = skol / 'github.com/piggyatbaqaqi'
drive.mount(str(content / "drive"), force_remount=True)
cache_path = content / 'cache'
ollama_cache_path = content / 'ollama_cache'
nb_path = content / 'packages_mistral'
if not os.path.exists(nb_path):
  nb_path.symlink_to(skol / 'packages_mistral')
skol_client = content / 'skol'
if not os.path.exists(skol_client):
  skol_client.symlink_to(piggyatbaqaqi / 'skol')
if not os.path.exists(cache_path):
  cache_path.symlink_to(skol / 'pip_cache')
if not os.path.exists(ollama_cache_path):
  ollama_cache_path.symlink_to(skol / 'ollama_cache')
os.environ['OLLAMA_MODELS'] = str(ollama_cache_path)
ist691 = skol_client / 'IST691'

sys.path.insert(0, str(nb_path))
sys.path.insert(0, str(piggyatbaqaqi / 'skol'))

/content
Mounted at /content/drive


In [85]:
# You only need to run this once per machine
try:
  import bitsandbytes
except ImportError:
  !pip install  --cache-dir=$cache_path --target=$nb_path bitsandbytes
try:
  import transformers
except ImportError:
  !pip install  --cache-dir=$cache_path --target=$nb_path transformers
try:
  import peft
except ImportError:
  !pip install  --cache-dir=$cache_path --target=$nb_path peft
try:
  import accelerate
except ImportError:
  !pip install  --cache-dir=$cache_path --target=$nb_path accelerate
try:
  import datasets
  import ipywidgets
  import fsspec
except ImportError:
  !pip install --cache-dir=$cache_path --target=$nb_path datasets scipy ipywidgets
  !pip install --cache-dir=$cache_path --target=$nb_path fsspec==2023.9.2


### Set up git clients

In [86]:
if not os.path.exists(piggyatbaqaqi):
  %mkdir -p $piggyatbaqaqi
if not os.path.exists(piggyatbaqaqi / 'skol'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/skol.git
sys.path.insert(0, piggyatbaqaqi / 'skol')
if not os.path.exists(piggyatbaqaqi / 'dr-drafts-mycosearch'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/dr-drafts-mycosearch.git
workdir = skol / 'IST691'
%cd $workdir

/content/drive/My Drive/SKOL/IST691


### Set up SKOL-specific code

In [87]:
from finder import read_files, parse_annotated, target_classes
from label import Label
from taxon import Taxon, group_paragraphs

SEED=12345
default_label = Label('Misc-exposition')
keep_labels = [Label('Description'), Label('Nomenclature')]

In [88]:
raw_directory_path = skol / 'raw_2025_02_05/'
ann_directory_path = skol / 'annotated_2025_02_27/journals'

In [89]:
# Function that reports all the txt files under a Google Drive folder path
def listFiles(folder: str) -> List[str]:
  # List all files in the folder
  try:
      files = [file for file in glob.glob(f'{folder}/**/*.txt*', recursive=True) if 'Sydowia' not in file]
      return files
  except FileNotFoundError:
      print(f"Folder '{folder}' not found.")
  except PermissionError:
      print(f"Permission denied to access folder '{folder}'.")

In [90]:
# check files in annotated directory
training_files = listFiles(ann_directory_path)
training_files[:10]

['/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol057/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol054/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s17.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s29.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s30.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s7.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s21.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s13.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s46.txt.ann']

In [91]:
paragraphs = list(parse_annotated(read_files(random.sample(training_files, 20))))
relabeled = list(target_classes(default=default_label, keep=keep_labels, paragraphs=paragraphs))

In [92]:
prompt = '''Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.
'''
prompt = '''Format the output as JSON.
Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
The innermost layer must be formated as lists of string-valued values..
Translate. if any, Latin paragraphs to English.
'''


description = """Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicalis rotundata;
cellula basalis cylindrica vel conico-truncata, ad basim 3.5–4.5 μm crassa; Appendicibus
lateralibus 0–2, brunneae, septata, cylindricae, surgentibus ex cellulla e apicem 2nd vel
3rd.

Anamorphic fungi. Colonies on natural substrate eﬀuse, black. Mycelium
superﬁcial, composed of branched, septate, pale brown to brown, smoothwalled hyphae, 1.5–3 μm thick. Conidiophores absent or short, 1–3-septate,
brown to dark brown, 11–28 × 4.5–5 μm. Conidiogenous cells monoblastic,
determinate, solitary, simple, lageniform or ampulliform, brown to dark brown,
smooth, 4.5–6.5 × 3.5–5 μm, 3–4.5 μm wide at the truncate apex. Conidial
secession schizolytic. Conidia holoblastic, solitary, acrogenous, straight or
curved, obclavate to obclavate-rostrate, dark brown to brown, smooth, 13–19distoseptate, 130–190 μm long, 7–9 μm thick in the broadest part, tapering
to 2–3 μm near the apex; apical cells rounded; basal cell cylindrical, truncate,
3.5–4.5 μm wide; lateral appendages 0–2, brown, septate, cylindrical, arising
from the 2nd or 3rd cells from the apex.
"""

In [93]:
def load_json_training(filename: str) -> List[Dict[str, Any]]:
  retval = []
  state = 'START'  # 'description', 'result'
  with open(filename, "r", encoding="utf-8") as file:
    lines = []
    description = ''
    for line in file:
      if line.startswith('Send to LLM:'):
        if state == "result":
          result = ''.join(lines)
          try:
            result_dict = json.loads(result)
          except json.JSONDecodeError as err:
            print(f'Err: {err}\n{result}')
          retval.append({'description': description, 'result': json.dumps(result_dict)})
        lines = []
        state = 'description'
      elif line.startswith('Result:'):
        if state == "description":
          description = ''.join(lines)
          lines = []
        state = 'result'
      else:
        lines.append(line)
    if state == 'result' and len(lines) > 0:
      result = ''.join(lines)
      try:
        result_dict = json.loads(result)
      except json.JSONDecodeError as err:
        print(f'Err: {err}\n{result}')
      retval.append({'description': description, 'result': json.dumps(result_dict)})
  return retval


In [94]:
json_training = load_json_training(workdir / 'json_training.txt')
print(json_training[0])

{'description': '\n\nSaprobic on stems. Ascomata, superﬁcial, thyriothecial; in section conical,\nrelatively small, opening with a minute ﬂat or slightly papillate ostiole. Asci 8spored, bitunicate, ﬁssitunicate, cylindrical or obclavate. Ascospores, 1-septate,\nlight brown.\nAnamorphs: None reported for the genus (Hyde et al. 2011).\n\n\n', 'result': '{"habit": ["saprobic"], "habitat": ["on stems"], "ascomata": ["superficial", "thyriothecial", "conical", "relatively small", "opening with a minute flat or slightly papillate ostiole"], "asci": {"number of spores": ["8-spored"], "shape": ["bitunicate", "fissitunicate", "cylindrical or obclavate"]}, "ascospores": {"septation": ["1-septate"], "color": ["light brown"]}, "anamorphs": {"presence": ["none reported for the genus (hyde et al. 2011)"]}}'}


In [95]:
import datasets

dataset = datasets.Dataset.from_list(json_training)

new_dataset = datasets.Dataset.train_test_split(dataset,int(1))
temp_dataset = new_dataset["train"]
test_dataset = new_dataset["test"]
new_dataset2 = datasets.Dataset.train_test_split(temp_dataset,int(1))
train_dataset = new_dataset2["train"]
eval_dataset = new_dataset2["test"]

print(train_dataset,eval_dataset,test_dataset)

Dataset({
    features: ['description', 'result'],
    num_rows: 14
}) Dataset({
    features: ['description', 'result'],
    num_rows: 1
}) Dataset({
    features: ['description', 'result'],
    num_rows: 1
})


In [96]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)


In [97]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [98]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token


In [99]:

def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [100]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""{prompt}

{data_point["description"]}
Result:
{data_point["result"]}
"""
    return tokenize(full_prompt)


In [101]:

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)
tokenized_test_dataset = test_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [102]:
print(tokenized_train_dataset[4]['input_ids'])
print(len(tokenized_train_dataset[4]['input_ids']))
print("\n" + test_dataset[0]['description'])
print("Result: " + test_dataset[0]['result'] + "\n")


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [103]:
eval_prompt = f"""{prompt}

{description}
Result:
"""



In [104]:
print(eval_prompt)

Format the output as JSON.
Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
The innermost layer must be formated as lists of string-valued values..
Translate. if any, Latin paragraphs to English.


Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicali

In [105]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=2048, pad_token_id=2)[0], skip_special_tokens=True))


Format the output as JSON.
Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
The innermost layer must be formated as lists of string-valued values..
Translate. if any, Latin paragraphs to English.


Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicali

In [106]:
from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )




In [107]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
print_trainable_parameters(model)
model = accelerator.prepare_model(model)



trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


In [108]:
print(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj)

In [109]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True


In [110]:
import transformers
from datetime import datetime
project = "skol-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name
tokenizer.pad_token = tokenizer.eos_token
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # max_steps=1000,
        max_steps=50,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=5,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=5,                # Save checkpoints every 50 steps
        eval_strategy="steps", # Evaluate the model every logging step
        eval_steps=5,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: piggy-yarroll (piggy-yarroll-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,0.860300,0.845157


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=0.8603462219238281, metrics={'train_runtime': 7629.6929, 'train_samples_per_second': 0.052, 'train_steps_per_second': 0.007, 'total_flos': 1.53364815937536e+16, 'train_loss': 0.8603462219238281, 'epoch': 25.0})

In [111]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [113]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(base_model, "mistral-skol-finetune/checkpoint-50")
ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))


Format the output as JSON.
Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
The innermost layer must be formated as lists of string-valued values..
Translate. if any, Latin paragraphs to English.


Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicali

In [114]:
ft_model.save_pretrained(str(ist691 / 'mistral-skol-finetune'))
tokenizer.save_pretrained(str(ist691 / 'mistral-skol-finetune'))

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('/content/skol/IST691/mistral-skol-finetune/tokenizer_config.json',
 '/content/skol/IST691/mistral-skol-finetune/special_tokens_map.json',
 '/content/skol/IST691/mistral-skol-finetune/tokenizer.model',
 '/content/skol/IST691/mistral-skol-finetune/added_tokens.json',
 '/content/skol/IST691/mistral-skol-finetune/tokenizer.json')

In [ ]:
prompt = '''Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.
'''
prompt = '''Format the output as JSON.
Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
The innermost layer must be formated as lists of string-valued values..
Translate. if any, Latin paragraphs to English.
'''


description = """Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicalis rotundata;
cellula basalis cylindrica vel conico-truncata, ad basim 3.5–4.5 μm crassa; Appendicibus
lateralibus 0–2, brunneae, septata, cylindricae, surgentibus ex cellulla e apicem 2nd vel
3rd.

Anamorphic fungi. Colonies on natural substrate eﬀuse, black. Mycelium
superﬁcial, composed of branched, septate, pale brown to brown, smoothwalled hyphae, 1.5–3 μm thick. Conidiophores absent or short, 1–3-septate,
brown to dark brown, 11–28 × 4.5–5 μm. Conidiogenous cells monoblastic,
determinate, solitary, simple, lageniform or ampulliform, brown to dark brown,
smooth, 4.5–6.5 × 3.5–5 μm, 3–4.5 μm wide at the truncate apex. Conidial
secession schizolytic. Conidia holoblastic, solitary, acrogenous, straight or
curved, obclavate to obclavate-rostrate, dark brown to brown, smooth, 13–19distoseptate, 130–190 μm long, 7–9 μm thick in the broadest part, tapering
to 2–3 μm near the apex; apical cells rounded; basal cell cylindrical, truncate,
3.5–4.5 μm wide; lateral appendages 0–2, brown, septate, cylindrical, arising
from the 2nd or 3rd cells from the apex.
"""

eval_prompt = f"""{prompt}

{description}
Result:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=2048, pad_token_id=2)[0], skip_special_tokens=True))
